# Area Processing (Tutorial 2)
***
# Table of Contents
1.   [Imports](#Imports)
2.   [Image Analysis](#Images-Analysis)
3.   [Exercise 1 - Sliding Window](#Exercise-1---Sliding-Window)
4.   [Exercise 2 - Convolution on RoI](#Exercise-2---Convolution-on-RoI)
5.   [Exercise 3 - Convolution on the Whole Image](#Exercise-3---Convolution-on-the-Whole-Image)
6.   [Exercise 4 - Different Convolution Kernels](#Exercise-4---Different-Convolution-Kernels)


# Imports

Only 3 libraries are needed for this project:
* opencv (cv2) - For image processing
* numpy - For its arrays
* matplotlib - Plotting histograms
* Classes - Custom classes written by me for this assignment

In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from Classes import Window, Sobel, Gaussian, Bilinear


# Image Analysis

### aiden dp.jpg

This is a picture of me in the upper barakka gardens in Valletta. The focus is myself with the background of the Grand
Harbour. The picture is lighted by natural lighting and includes a lot of non linear features.

<img src="images/aiden dp.jpg">

### cursed.jpg

The image I will be using for this lab is one of my failed attempts of creating a pie chart for one of my statistics units
from last year. It features a sun like object in the centre with black lines coming out from the centre, it is coloured in
full RGB while the background is gray.

<img src="images/cursed.jpg">

### dog chaos.png

This is a blurry picture of two of my friends and another friend's dog. They are in a dimly lit, tiled floor room with a
white wall and some furniture behind one of my friends.

<img src="images/dog chaos.png">

### jake car.png

This is a picture of my classmate Jake standing next to a red car. The photo was taken in a street during daytime. The
background features several home facades, 2 other cars and a person.

<img src="images/jake car.png">

### jake close up.jpg

This is a picture of my classmate Jake with a beige background.

<img src="images/jake close up.jpg">

### jake sitting.jpg

This is a picture of my classmate Jake sitting on the floor of a bathroom. The room is well lit and tiled all over.

<img src="images/jake sitting.jpg">

### jojo ben.jpg

This is a picture of my classmate Ben walking towards the Hal Ghaxaq church with a bag of fried chicken in his hand. The
photo was taken during the night so the lighting comes from old street lamps.

<img src="images/jojo ben.jpg">

#### I have permission by all the people shown to use these images for this tutorial

# Exercise 1 - Sliding Window

In [4]:
%%capture
win = Window(gray, 500, 100)
start_point, end_point = win.getPos()

# White
color = (255, 255, 255)

image = cv2.rectangle(gray.copy(), start_point, end_point, color)

cv2.imwrite("Output/GrayWithRectangle.png", image, [cv2.IMWRITE_PNG_COMPRESSION, 0])
image2 = win.getImageInBoundary(gray)
cv2.imwrite("Output/ROI.png", image2, [cv2.IMWRITE_PNG_COMPRESSION, 0])

In [5]:
%%capture
show = False

new_tl, new_br = win.forwardPos()
while win.inBoundary(new_br):
    image = cv2.rectangle(gray.copy(), start_point, end_point, (255, 255, 255))
    if show:
        cv2.imshow("Sliding Window", image)
        cv2.waitKey(int(1/35*1000))
    start_point, end_point = win.forwardMove()
    new_tl, new_br = win.forwardPos()
cv2.destroyAllWindows()
cv2.imwrite("Output/LastPic.png", image, [cv2.IMWRITE_PNG_COMPRESSION, 0])


# Exercise 2 - Convolution on RoI

In [6]:
# sobel = Sobel()
#
# win = Window(gray, 300, 1)
#
# image = cv2.imread("1mb pic.png")
# image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#
# roi = win.getImageInBoundary(image)
# cv2.imwrite("Output/roi_before_filter.png", roi, [cv2.IMWRITE_PNG_COMPRESSION, 0])
# filtered = sobel.filterImage(image, win)
# cv2.imwrite("Output/roi_after_filter.png", filtered, [cv2.IMWRITE_PNG_COMPRESSION, 0])


IndexError: index 300 is out of bounds for axis 0 with size 300

# Exercise 3 - Convolution on the Whole Image

# Exercise 4 - Different Convolution Kernels
